In [15]:
import datetime
import Image
import gc
import numpy as np
import os
import random
from scipy import misc
import string
import time

# Set some Theano config before initializing
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=cpu,floatX=float32,allow_gc=False,openmp=True"
import theano

import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt

#import emotion_model
import dwdii_bc_model_helper as bc
import bc_models as models

random.seed(20275)

In [16]:
print "device:", theano.config.device
print "floatX:",  theano.config.floatX
print "mode:", theano.config.mode
print "openmp:", theano.config.openmp
print "allow_gc:", theano.config.allow_gc

device: cpu
floatX: float32
mode: FAST_RUN
openmp: True
allow_gc: False


In [23]:
imagePath = "/root/bc_data/ddsm-png.25"
dataPath = "/root/bc_data/ddsm-png.25/Ddsm_png.csv"
imgResize = (150, 150)

In [24]:
os.listdir('/root/bc_data')

['ddsm', 'ddsm-hog', 'ddsm-png.25', 'ddsm-sm', 'Ddsm.csv', 'mias']

# Load Training and Test Data

In this section, the training/validation data is loaded. The load_data function pre-balances the data set by removing images from over-represented emotion classes.

In [25]:
metaData = bc.load_training_metadata(dataPath, verbose=True)

Before Balancing
----------------
benign 3164
malignant 1096

After Balancing
----------------
benign 3164
malignant 1096


In [6]:
# TODO: Actually load some representative data for model experimentation
maxData = 1000
X_data, Y_data = bc.load_data(dataPath, imagePath, maxData = maxData, verboseFreq = 50, imgResize=imgResize)
print X_data.shape
print Y_data.shape

0.000000: A_0048_1.RIGHT_CC.LJPEG.pgm
0.113636: A_0037_1.RIGHT_MLO.LJPEG.pgm
0.227273: A_0026_1.RIGHT_CC.LJPEG.pgm
0.340909: A_0049_1.LEFT_MLO.LJPEG.pgm
0.454545: A_0012_1.LEFT_CC.LJPEG.pgm
0.568182: A_0037_1.RIGHT_CC.LJPEG.pgm
0.681818: A_0032_1.LEFT_CC.LJPEG.pgm
0.795455: A_0057_1.LEFT_MLO.LJPEG.pgm
0.909091: A_0024_1.LEFT_CC.LJPEG.pgm
(176, 150, 150)
(176, 1)


## Split Training/Test Sets
The following code segment splits the data into training and test data sets. Currently this is a standard 80/20 split for training and test respectively after performing a random shuffle using the unison_shuffled_copies help method.

In [7]:
skippedTransforms = True
if skippedTransforms:
    X_data2 = X_data
    Y_data2 = Y_data

In [8]:
# Split the data into Training and Test sets
trainNdx = int(X_data2.shape[0] * .8)
print trainNdx

X_train, X_test = np.split(X_data2, [trainNdx])
Y_train, Y_test = np.split(Y_data2, [trainNdx])
print X_train.shape
print X_test.shape

print Y_train.shape
print Y_test.shape

140
(140, 150, 150)
(36, 150, 150)
(140, 1)
(36, 1)


## Define and Load Trained Model

In [9]:
# Load the bc array for our count in the model definition
bcTypes = bc.bcNumerics()
print bcTypes
print len(bcTypes)

{'benign': 0, 'malignant': 1}
2


In [11]:
# Construct the model using our help function
model = models.bc_model_v0(len(bcTypes), verbose=True, 
                                        input_shape=(1,X_train.shape[1],X_train.shape[2]))

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
convolution2d_5 (Convolution2D)    (None, 32, 143, 143)2080        convolution2d_input_2[0][0]      
____________________________________________________________________________________________________
activation_7 (Activation)          (None, 32, 143, 143)0           convolution2d_5[0][0]            
____________________________________________________________________________________________________
maxpooling2d_5 (MaxPooling2D)      (None, 32, 71, 71)  0           activation_7[0][0]               
____________________________________________________________________________________________________
convolution2d_6 (Convolution2D)    (None, 32, 67, 67)  25632       maxpooling2d_5[0][0]             
___________________________________________________________________________________________

## Training the Model

The following code segment trains the model using the run_network helper function. 

In [12]:
loadWeights = False
if loadWeights:
    model.load_weights("dwdii-bc-150-v0-Cloud.hdf5")

In [13]:
# Reshape to the appropriate shape for the CNN input
testX = X_test.reshape(X_test.shape[0], 1, X_train.shape[1],X_train.shape[2])
trainX = X_train.reshape(X_train.shape[0], 1, X_train.shape[1],X_train.shape[2])

In [14]:
print "Training start: " + str(datetime.datetime.now())
m, h = models.run_network([trainX, testX, Y_train, Y_test], model, batch=20, epochs=30, verbosity=1)

Training start: 2017-03-19 21:24:15.741023
(140, 1)
(36, 1)
Training model...


Exception: A target array with shape (140, 1) was passed for an output of shape (None, 2) while using as loss `binary_crossentropy`. This loss expects targets to have the same shape as the output.

In [ ]:
model.save_weights("dwdii-bc-150-v0-Cloud.hdf5", overwrite=True)